# Roteiro de trabalho

### Tempos dos vídeos
- (IT) Insulina Tradicional (15:00) $\rightarrow$ 900 segundos
- (GT) Glucagon Tradicional (14:57) $\rightarrow$ 897 segundos
- (IA) Insulina Ativa (19:26) $\rightarrow$ 1166 segundos
- (GA) Glucagon Ativa (20:43) $\rightarrow$ 1243 segundos

### Participante deste roteiro:
- Nathalia: IA e GT

In [1]:
import sys
import mne
import numpy as np
from scipy.signal import welch
import matplotlib.pyplot as plt
from matplotlib import rcParams

#### Definição do objeto MNE info

In [2]:
n_channels = 8
ch_types = ['eeg'] * n_channels
sfreq = 250
ch_names = ["F3", "Fz", "F4", "C3", "Cz", "C4", "P3", "P4"]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
info.set_montage("standard_1020")

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


#### Carregamento dos dados e transformação para objeto MNE RawArray

In [3]:
# Essa minutagem é sobre GT ou IA? qual(is) arquivo(s)? concateno os arquivos?
# Assumindo que seja sobre aula de GT
# inicio: 30s
# Iniciou teste: 15:32
# Final: 18:08

# Aula GT: 7500 até 232750 [225250 segundos]
# Mini Teste GT: 233000 até 272000 [39000 segundos]
gt = '../dataset-s6/OpenBCI-RAW-2023-10-10_14-23-56.txt'
gt_ob = np.loadtxt(gt, delimiter=',', skiprows=5, usecols=range(1, 9))

# Removendo trechos inúteis (pré-início e pós-fim)
data_gt = gt_ob[7500:232750, :]
data_gt_test = gt_ob[233000:272000, :]

X = {
    'gt': mne.io.RawArray(data_gt.T, info),
    'gt_test': mne.io.RawArray(data_gt_test.T, info),
}

Creating RawArray with float64 data, n_channels=8, n_times=225250
    Range : 0 ... 225249 =      0.000 ...   900.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=39000
    Range : 0 ... 38999 =      0.000 ...   155.996 secs
Ready.


#### Filtragem dos dados

Primeiramente, atenua-se a frequência de 60Hz evitando a interferência da rede elétrica e, posteriormente, obtem-se a faixa de frequência desejada de 4 a 100Hz.

In [1]:
for key in X:
    X[key].notch_filter(freqs=60)
    X[key].filter(l_freq=4, h_freq=100)

NameError: name 'X' is not defined

#### Janela Deslizante

O trecho a seguir determina a porcentagem do tempo em que cada banda de frequência é dominante ao longo de uma janela de tempo específica.

In [5]:
sr = 250            # Taxa de amostragem dos dados do EEG (250Hz)
jump = 5            # Tamanho da janela de análise (5 em 5 segundos)
size = sr * jump    # Número de pontos de dados em cada janela de análise

# Definindo os limites das bandas de frequência (em Hz)
theta_band = (4, 8)       # Theta: 4 - 8 Hz
alpha_band = (8, 13)      # Alpha: 8 - 13 Hz
beta_band = (13, 30)      # Beta: 13 - 30 Hz
gamma_band = (30, 100)    # Gamma: 30 - 100 Hz


total_pc = {}
data_names = ('gt', 'gt_test')

# X contém os dois conjuntos de dados de EEG para análise (X['gt'] e X['gt_test'])
for k, data in enumerate(X.values()):
    results = [0, 0, 0, 0] # Contagem de dominância de cada banda
    
    # Percorre os dados em incrementos de sr (1 segundo) extraindo janelas de tamanho size (5s)
    for i in range(0, len(data)-1, sr):
        cut = data.get_data(start=i, stop=i+size)

        # Aplicando o filtro CAR
        mean_reference = np.mean(cut, axis=0)  # Calculando a média comum de referência
        car_filtered_data = cut - mean_reference  # Subtraindo a média de cada canal

        # Número de pontos por segmento
        nperseg = 128
        # Quantidade de sobreposição entre segmentos
        noverlap = nperseg // 2 # 64
        
        freqs, psd = welch(car_filtered_data, fs=sr, nperseg=nperseg, noverlap=noverlap)
        
        # realizando a média dos 8 eletrodos
        # set_eeg_reference (MNE)
        X = np.average(psd, axis=0)

        # Encontrar os índices correspondentes às frequências de interesse
        theta_idxs = np.where((freqs >= theta_band[0]) & (freqs <= theta_band[1]))[0]
        alpha_idxs = np.where((freqs >= alpha_band[0]) & (freqs <= alpha_band[1]))[0]
        beta_idxs = np.where((freqs >= beta_band[0]) & (freqs <= beta_band[1]))[0]
        gamma_idxs = np.where((freqs >= gamma_band[0]) & (freqs <= gamma_band[1]))[0]

        # Calculando a potência em cada banda de frequência por meio da integração da PSD
        bands = [np.sum(X[theta_idxs]), np.sum(X[alpha_idxs]), np.sum(X[beta_idxs]), np.sum(X[gamma_idxs])]
        # Calculando a pontuação de cada banda para mostrar a porcentagem futura
        results[np.argmax(bands)] += 1

    total = sum(results)
    percentages = [round((count/total) * 100, 2) for count in results]
    print(data.__str__())
    total_pc[data_names[k]] = percentages

print(total_pc)

<RawArray | 8 x 225250 (901.0 s), ~13.8 MB, data loaded>
[96.45, 0.0, 0.22, 3.33]
<RawArray | 8 x 39000 (156.0 s), ~2.4 MB, data loaded>
[94.23, 0.0, 0.0, 5.77]


Mostrando os gráficos de barra dos resultados.

In [7]:
rcParams['figure.figsize'] = [6., 4.]

def plot_bar(x):
    colors = ('orange', 'green', 'blue', 'purple')
    bars = plt.bar(('theta', 'alpha', 'beta', 'gamma'), x, color=colors)
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')
    plt.title('Porcentagem da potência em cada banda de frequência')
    plt.xlabel('Banda de Frequência')
    plt.ylabel('Porcentagem durante a atividade')
    plt.show()


Aplicando os valores de porcentagens obtidos na função de gráfico

In [ ]:
print('Glucagon Tradicional')
plot_bar(total_pc['gt'])
print('Glucagon Tradicional - Teste')
plot_bar(total_pc['gt_test'])

#### Tarefa 1 - Aplicando um Filtro Espacial nos Dados:

Realizando a média dos 8 eletrodos, analizar os canais separadamente. Para isso, vamos aplicar o filtro espacial, no qual   podemos utilizar algoritmos específicos para a análise de EEG utilizando o CAR (Common Average Reference). Na biblioteca MNE existe o método `set_eeg_reference`, que pode ser utilizado para aplicar o filtro espacial.

Para aplicar o `set_eeg_reference`, é necessário que os dados estejam no objeto `RawArray`, que é o objeto que contém o método.

O slice deve ser obtido diretamente do objeto MNE, tendo que nesse caso modificar todo o laço de controle.

`cut = data.get_data(start=i, stop=i+size)`

#### Tarefa 2 - Análise de Recorte Específico

Observar por meio dos códigos, os 3 principais rítmos de interesse (Theta, Beta e Gama), com tempos mais específicos do experimento. Ou seja, até o momento, observamos as porcentagens olhando para todo experimento. Então mesmo obtendo um valor de Beta às vezes baixo, em algum momento considerável da aula assistida, ele pode estar alto. Faça uma análise mais detalhada desses momentos.

Para tal, descobra os segmentos de interesse através da analise dos dados diretamente no software OpenBCI GUI. Posteriormente, pegue as mesmas faixas de tempo, e realize os recortes no código em python para constar a prova de conceito.

Mostre também gráficos no domínio da frequência e/ou espectrograma.